In [1]:
import tensorflow        as tf
import numpy             as np
import matplotlib.pyplot as plt

from tensorflow.python.framework import ops
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import cv2
import os
import copy

from random import shuffle

In [2]:
training_path = 'D:/imagenet_object_localization/ILSVRC/Data/CLS-LOC/train/'

In [3]:
#writer = tf.summary.FileWriter('./log_tmp/', sess.graph)

In [4]:
def get_label_dict(file_path):
    key_to_label = {}
    keys         = []
    
    pos            = 0
    key_length     = 9
    newline_length = 1
    
    with open(file_path) as file:
        for index in range(1000):
            key = file.read(key_length)
        
            file.seek(pos + key_length + newline_length)
            
            label = file.readline()
            
            file.seek(pos + key_length + newline_length + len(label))
            
            pos = file.tell()
            
            label             = label.replace("\n", "")
            key_to_label[key] = label
            
            keys.append(key)

    return keys, key_to_label

In [5]:
def load_image(filePath, training=True):
    image = cv2.imread(filePath)
    
    if training:
        image = cv2.resize(image, (224, 224))
        
    return image

In [6]:
def get_training_set(folder_path, size=-1):
    training_examples = []
    count = 0
    for image_set in os.listdir(folder_path):
        for image in os.listdir(folder_path + image_set):
            example = { 'label' : image_set, 'path' : folder_path + image_set + '/' + image}
            training_examples.append(example)
            count += 1
            
        if size != -1 and count > size:
             break
            
    return training_examples

In [7]:
def get_training_batch(training_examples, batch, batch_size, onehot_encoded, label_encoder):
    start = batch * batch_size
    end   = (batch + 1) * batch_size
    
    labels = []
    images = []
    
    for index in range(start, end):
        example = training_examples[index]
        image = cv2.imread(example['path'])
        image = cv2.resize(image, (224, 224))
        
        onehot_label = onehot_encoded[label_encoder.transform([example['label']])[0]]
        
        labels.append(onehot_label)
        images.append(image       )
        
    return labels, images

In [8]:
def start_model(model_input):
    output = tf.layers.conv2d(model_input, filters=64, kernel_size=(7,7), strides=(2,2))
    output = tf.layers.batch_normalization(output)
    output = tf.nn.relu(output)
    output = tf.layers.max_pooling2d(output, pool_size=(3,3) ,strides=(2,2))
    
    return output

In [9]:
def skip_connection(input_layer, filters, layer):
    x      = tf.layers.conv2d             (input_layer, filters=filters, kernel_size=(3, 3), padding='SAME', name="conv1_" + layer)
    output = tf.layers.batch_normalization(x, name="batch1_" + layer)
    output = tf.nn.relu                   (output, name="relu1_" + layer)
    
    output = tf.layers.conv2d             (output     , filters=filters, kernel_size=(3, 3), padding='SAME', name="conv2_" + layer)
    output = tf.layers.batch_normalization(output, name="batch2_" + layer)
    output = tf.nn.relu                   (output, name="relu2_" + layer)
    
    skip   = tf.layers.conv2d(x     ,filters=filters , kernel_size=(1, 1), strides=(1,1), padding='SAME', activation=tf.nn.relu, name="convskip_" + layer)
    skip   = tf.nn.relu(skip, name="reluskip_" + layer)
    
    return tf.keras.layers.concatenate([output, skip])

In [10]:
def get_resnet_model(num_categories = 1000, gpu=False):
    model_input = tf.placeholder(dtype = tf.float32, shape = (None, 224, 224, 3), name = 'model_input')
    is_training = tf.placeholder(dtype = tf.bool,                                 name = 'is_training')
    
    device_string = "gpu:0" if gpu else "cpu:0"
    
    with tf.device(device_string):
        output = start_model(model_input)
        
        #FIRST SET OF 64 FILTER CONVOLUTIONS
        output = skip_connection(output, 64,  "1a")
        output = skip_connection(output, 64,  "1b")
        output = skip_connection(output, 64,  "1c")
        output = skip_connection(output, 64,  "1d")
        output = skip_connection(output, 64,  "1e")
        output = skip_connection(output, 64,  "1f")
        
        #SECOND SET OF 128 FILTER CONVOLUTIONS
        output = skip_connection(output, 224, "2a")
        output = skip_connection(output, 224, "2b")
        output = skip_connection(output, 224, "2c")
        output = skip_connection(output, 224, "2d")
        output = skip_connection(output, 224, "2e")
        output = skip_connection(output, 224, "2f")
        output = skip_connection(output, 224, "2g")
        output = skip_connection(output, 224, "2h")
        
        #THIRD SET OF 256 FILTER CONVOLUTIONS
        output = skip_connection(output, 224, "3a")
        output = skip_connection(output, 224, "3b")
        output = skip_connection(output, 224, "3c")
        output = skip_connection(output, 224, "3d")
        output = skip_connection(output, 224, "3e")
        output = skip_connection(output, 224, "3f")
        output = skip_connection(output, 224, "3g")
        output = skip_connection(output, 224, "3h")
        output = skip_connection(output, 224, "3i")
        output = skip_connection(output, 224, "3j")
        output = skip_connection(output, 224, "3k")
        output = skip_connection(output, 224, "3l")
        
        #FOURTH SET OF 512 FILTER CONVOLUTIONS
        output = skip_connection(output, 224, "4a")
        output = skip_connection(output, 224, "4b")
        output = skip_connection(output, 224, "4c")
        output = skip_connection(output, 224, "4d")
        output = skip_connection(output, 224, "4e")
        output = skip_connection(output, 224, "4f")
        
        #AVERAGE POOL
        output = tf.layers.average_pooling2d(output, pool_size=(2,2), strides=(2,2), name="avg_pool")
        
        #FLATTEN AND PERFORM SOFTMAX
        output = tf.layers.flatten(output)
        output = tf.layers.dense  (output, num_categories)
        
        #IF WE'RE TRAINING, ALLOW COST LAYER TO PERFORM SOFTMAX, ELSE IF INFERING PEROFRM IT HERE
        output = tf.cond(is_training, lambda: output, lambda: tf.nn.softmax(output))
        
        output = tf.identity(output, name="final_output")
    
    return output, model_input, is_training

In [11]:
def get_key_encoder(keys):
    label_encoder   = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(keys)
    
    onehot_encoder  = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded  = onehot_encoder.fit_transform(integer_encoded)
    
    return label_encoder, onehot_encoded

#label_encoder, onehot_encoded = get_key_encoder(keys)

#print(onehot_encoded[label_encoder.transform(['n01484850'])[0]])

In [ ]:
t = get_label_dict("D:/imagenet_object_localization/LOC_synset_mapping/LOC_synset_mapping.txt")

In [20]:
ops.reset_default_graph()

random_input= np.random.rand(256, 224, 224, 3)

model, model_input, is_training = get_resnet_model(gpu=True)

init  = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    skip = sess.run([model], feed_dict = { model_input : random_input, is_training : True})
    
print(skip[0].shape)

ResourceExhaustedError: OOM when allocating tensor with shape[326592,1000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node dense/kernel/Initializer/random_uniform/RandomUniform (defined at <ipython-input-10-483599925f48>:55)  = RandomUniform[T=DT_INT32, _class=["loc:@dense/kernel/Assign"], dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense/kernel/Initializer/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense/kernel/Initializer/random_uniform/RandomUniform', defined at:
  File "c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\Lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\Lib\runpy.py", line 85, in _run_code
    exec(code, run_globals

In [ ]:
keys, key_dict = get_label_dict('D:/imagenet_object_localization/LOC_synset_mapping/LOC_synset_mapping.txt')

In [12]:
def train_resnet(training_examples, keys, learning_rate=1e-5, num_epochs=5, batch_size=16, epoch_start=0, restore_path=""):
    ops.reset_default_graph()
    
    m                             = len(training_examples)
    label_encoder, onehot_encoded = get_key_encoder(keys)
    
    labels = tf.placeholder(dtype = tf.float32, shape = (None, 1000), name = 'labels')
    
    model, model_input, is_training = get_resnet_model(gpu=True)
    
    with tf.device('gpu:0'):
        cost      = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=model)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
        
    saver = tf.train.Saver()
    init  = tf.global_variables_initializer()
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init)
    
    if os.path.exists(restore_path) == True:
        saver = tf.train.import_meta_graph(restore_path + 'resnet.ckpt.meta')
        saver.restore(sess, restore_path + 'resnet.ckpt')
        
        
    tf.train.write_graph(sess.graph.as_graph_def(), './train/resnet_model', 'resnet.pbtxt', True)
    
    for epoch in range(epoch_start, epoch_start + num_epochs):
        epoch_cost  = 0
        num_batches = (int)(m / batch_size)
    
        shuffle(training_examples)
        
        for batch in range(num_batches):
            #l, i           = get_training_batch(training_examples, batch, batch_size, onehot_encoded, label_encoder)
            l = np.random.rand(16, 1000)
            i = np.random.rand(16, 224, 224, 3)
            
            
            
            _, batch_cost  = sess.run([optimizer, cost], feed_dict = { model_input : i, labels : l, is_training : True})
            epoch_cost    += batch_cost / num_batches
            
            print(str(batch) + ' out of ' + str(num_batches))
                    
        if epoch % 10 == 0:
            saver.save(sess, './train_' + str(epoch) + '/resnet.ckpt')
            print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            print ()
            print ("---------------------------------------------")
            print ()
        
    print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
    print ()
    print ("---------------------------------------------")
    print ()
    saver.save(sess, './train_final/restnet.ckpt')

In [13]:
keys, key_dict = get_label_dict('D:/imagenet_object_localization/LOC_synset_mapping/LOC_synset_mapping.txt')

In [14]:
training_set = get_training_set(training_path, size=1000)

shuffle(training_set)

In [13]:
len(training_set)

1300

In [ ]:
training_set[1]

In [15]:
train_resnet(training_set, keys)

ResourceExhaustedError: OOM when allocating tensor with shape[16,224,54,54] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node conv2_2a/Conv2D (defined at <ipython-input-9-3da0f65c3867>:6)  = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](relu1_2a, conv2_2a/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node softmax_cross_entropy_loss/value/_11}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_9603_softmax_cross_entropy_loss/value", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a